Importing Libraries

In [3]:

import numpy as np 

import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows',None)

import json

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

from bs4 import BeautifulSoup
import lxml

# import k-means from clustering stage
from sklearn.cluster import KMeans
from sklearn.datasets.samples_generator import make_blobs

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    geopy-1.20.0               |             py_0          57 KB  conda-forge
    openssl-1.1.1c             |       h516909a_0         2.1 MB  conda-forge
    certifi-2019.9.11          |           py36_0         147 KB  conda-forge
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    ca-certificates-2019.9.11  |       hecc5488_0         144 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         2.5 MB

The following NEW packages will be INSTALLED:

    geographiclib:   1.50-py_0         conda-forge
    geopy:           1.20.0-py_0       conda-forge

The following packages will be UPDATED:

    ca-

In [7]:
# Data scraping from wikipedia

raw_data = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
soup = requests.get(raw_data).text
scraped_data = BeautifulSoup(soup, 'lxml')

In [15]:
# Creating new data frame and transforming the data to df

column_names = ['Postalcode','Borough','Neighborhood']
canada_data = pd.DataFrame(columns = column_names)
canada_data

,Postalcode,Borough,Neighborhood


In [46]:
#Loop the data and updating the data frame one row at a time

content = scraped_data.find('div', class_='mw-parser-output')
table = content.table.tbody
postcode = 0
borough = 0
neighborhood = 0
table

for tr in table.find_all('tr'):
    i = 0
    for td in tr.find_all('td'):
        if i == 0:
            postcode = td.text
            i = i + 1
        elif i == 1:
            borough = td.text
            i = i + 1
        elif i == 2: 
            neighborhood = td.text.strip('\n').replace(']','')
            canada_data = canada_data.append({'Postalcode': postcode,'Borough': borough,'Neighborhood': neighborhood},ignore_index=True)

In [25]:
canada_data = canada_data[canada_data.Borough!='Not assigned']
canada_data = canada_data[canada_data.Borough!= 0]
canada_data.reset_index(drop = True, inplace = True)
i = 0
for i in range(0,canada_data.shape[0]):
    if canada_data.iloc[i][2] == 'Not assigned':
        canada_data.iloc[i][2] = canada_data.iloc[i][1]
        i = i+1
                                 
df = canada_data.groupby(['Postalcode','Borough'])['Neighborhood'].apply(', '.join).reset_index()
df.head()

,Postalcode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern, Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union, Highla..."
2,M1E,Scarborough,"Guildwood, Morningside, West Hill, Guildwood, ..."
3,M1G,Scarborough,"Woburn, Woburn"
4,M1H,Scarborough,"Cedarbrae, Cedarbrae"


In [51]:
# Data Cleaning

df = df.drop(df[(df.Borough == "Not assigned")].index) # drop rows if the borogh value is not assigned
df.Neighborhood.replace("Not assigned", df.Borough, inplace=True) # if neighborhood is not ssigned ten assign borough
df.Neighborhood.fillna(df.Borough, inplace=True)
df=df.drop_duplicates()

In [38]:
def grp_neighborhood(grouped):
        return ', '.join(sorted(grouped['Neighborhood'].tolist()))
                    
grp_attr = df.groupby(['Postalcode', 'Borough'])
df1 = grp_attr.apply(grp_neighborhood).reset_index(name='Neighborhood')

In [55]:
df1.head()

,Postalcode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern, Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union, Highla..."
2,M1E,Scarborough,"Guildwood, Morningside, West Hill, Guildwood, ..."
3,M1G,Scarborough,"Woburn, Woburn"
4,M1H,Scarborough,"Cedarbrae, Cedarbrae"


In [56]:
print(df1.shape)

(103, 3)


In [49]:
# geo data

geo_data = pd.read_csv("http://cocl.us/Geospatial_data")
geo_data.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [66]:
# Merging geo data to the data frame
df_geo_data = df1.join(geo_data.set_index('Postal Code'), on='Postalcode')
df_geo_data

,Postalcode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern, Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union, Highla...",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill, Guildwood, ...",43.763573,-79.188711
3,M1G,Scarborough,"Woburn, Woburn",43.770992,-79.216917
4,M1H,Scarborough,"Cedarbrae, Cedarbrae",43.773136,-79.239476
5,M1J,Scarborough,"Scarborough Village, Scarborough Village",43.744734,-79.239476
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park, E...",43.727929,-79.262029
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge, Clairlea, Gol...",43.711112,-79.284577
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village Wes...",43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff, Cliffside West, Birch Cliff, Clif...",43.692657,-79.264848


In [62]:
#Finding coordinates of Toronto

address = 'Toronto, ON'

geolocator = Nominatim(user_agent="on_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of toronto are 43.653963, -79.387207.


In [65]:
# Creating map of toronto
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

for lat, lng, borough, neighborhood in zip(df_geo_data['Latitude'], df_geo_data['Longitude'], df_geo_data['Borough'], df_geo_data['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [69]:
# slicing the central toronto data and creating a new data frame

Central_Toronto_data = df_geo_data[df_geo_data['Borough'] == 'Central Toronto'].reset_index(drop=True)
Central_Toronto_data.head()


,Postalcode,Borough,Neighborhood,Latitude,Longitude
0,M4N,Central Toronto,"Lawrence Park, Lawrence Park",43.728020,-79.388790
1,M4P,Central Toronto,"Davisville North, Davisville North",43.712751,-79.390197
2,M4R,Central Toronto,"North Toronto West, North Toronto West",43.715383,-79.405678
3,M4S,Central Toronto,"Davisville, Davisville",43.704324,-79.388790
4,M4T,Central Toronto,"Moore Park, Summerhill East, Moore Park, Summe...",43.689574,-79.383160


In [70]:
#Finding coordinates of  Central Toronto

address = 'Central Toronto, ON'

geolocator = Nominatim(user_agent="on_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Central Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Central Toronto are 43.653963, -79.387207.


In [71]:
# Foursquare Credentials and Version

CLIENT_ID = 'VCE3BFI5H3LYTHSZKAY1WEEBUGJCDTIFTAXVJHFDZVEBNW0D' # your Foursquare ID
CLIENT_SECRET = 'FYIMB3PO5KR3JJQ4O1ZQRJPAZNWZAZCWUL4U3FZDWU5LXPYF' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: VCE3BFI5H3LYTHSZKAY1WEEBUGJCDTIFTAXVJHFDZVEBNW0D
CLIENT_SECRET:FYIMB3PO5KR3JJQ4O1ZQRJPAZNWZAZCWUL4U3FZDWU5LXPYF


In [72]:
# identifying central toronto neighborhood 

Central_Toronto_data.loc[0, 'Neighborhood']

'Lawrence Park, Lawrence Park'

In [73]:
# identifying neighborhood latitude and longitude

neighborhood_latitude = Central_Toronto_data.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = Central_Toronto_data.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = Central_Toronto_data.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Lawrence Park, Lawrence Park are 43.7280205, -79.3887901.


In [74]:
# venues in central toronto neighborhood

LIMIT = 100 # no of venues

radius = 500 

url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url


'https://api.foursquare.com/v2/venues/explore?&client_id=VCE3BFI5H3LYTHSZKAY1WEEBUGJCDTIFTAXVJHFDZVEBNW0D&client_secret=FYIMB3PO5KR3JJQ4O1ZQRJPAZNWZAZCWUL4U3FZDWU5LXPYF&v=20180605&ll=43.7280205,-79.3887901&radius=500&limit=100'

In [75]:
#Examining the results of GET request

results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5daeac4aa4b51b66a78821c3'},
 'response': {'warning': {'text': "There aren't a lot of results near you. Try something more general, reset your filters, or expand the search area."},
  'headerLocation': 'Toronto',
  'headerFullLocation': 'Toronto',
  'headerLocationGranularity': 'city',
  'totalResults': 3,
  'suggestedBounds': {'ne': {'lat': 43.7325205045, 'lng': -79.3825744605273},
   'sw': {'lat': 43.7235204955, 'lng': -79.3950057394727}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '50e6da19e4b0d8a78a0e9794',
       'name': 'Lawrence Park Ravine',
       'location': {'address': '3055 Yonge Street',
        'crossStreet': 'Lawrence Avenue East',
        'lat': 43.72696303913755,
        'lng': -79.39438246708775,
        'labeledL

In [76]:
# function that extracts the category of the venue


def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [80]:
# structuring the venues into a data frame

venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Lawrence Park Ravine,Park,43.726963,-79.394382
1,Zodiac Swim School,Swim School,43.728532,-79.382860
2,TTC Bus #162 - Lawrence-Donway,Bus Line,43.728026,-79.382805


In [81]:
print(nearby_venues.shape)

(3, 4)


In [82]:
# all the near by central toronto venues

def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [85]:
central_toronto_venues = getNearbyVenues(names=Central_Toronto_data['Neighborhood'],
                                   latitudes=Central_Toronto_data['Latitude'],
                                   longitudes=Central_Toronto_data['Longitude']
                                  )

Lawrence Park, Lawrence Park
Davisville North, Davisville North
North Toronto West, North Toronto West
Davisville, Davisville
Moore Park, Summerhill East, Moore Park, Summerhill East
Deer Park, Forest Hill SE, Rathnelly, South Hill, Summerhill West, Deer Park, Forest Hill SE, Rathnelly, South Hill, Summerhill West
Roselawn, Roselawn
Forest Hill North, Forest Hill West, Forest Hill North, Forest Hill West
The Annex, North Midtown, Yorkville, The Annex, North Midtown, Yorkville


In [87]:
#Exploring the central_toronto_venues data frame

print(central_toronto_venues.shape)

central_toronto_venues.head()

(118, 7)
There are 65 uniques categories.


In [89]:
central_toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Davisville North, Davisville North",8,8,8,8,8,8
"Davisville, Davisville",39,39,39,39,39,39
"Deer Park, Forest Hill SE, Rathnelly, South Hill, Summerhill West, Deer Park, Forest Hill SE, Rathnelly, South Hill, Summerhill West",15,15,15,15,15,15
"Forest Hill North, Forest Hill West, Forest Hill North, Forest Hill West",4,4,4,4,4,4
"Lawrence Park, Lawrence Park",3,3,3,3,3,3
"Moore Park, Summerhill East, Moore Park, Summerhill East",3,3,3,3,3,3
"North Toronto West, North Toronto West",22,22,22,22,22,22
"Roselawn, Roselawn",2,2,2,2,2,2
"The Annex, North Midtown, Yorkville, The Annex, North Midtown, Yorkville",22,22,22,22,22,22


In [88]:
print('There are {} uniques categories.'.format(len(central_toronto_venues['Venue Category'].unique())))

There are 65 uniques categories.


In [91]:
# Analyzing each neighborhood

# one hot encoding
Toronto_onehot = pd.get_dummies(central_toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
Toronto_onehot['Neighborhood'] = central_toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [Toronto_onehot.columns[-1]] + list(Toronto_onehot.columns[:-1])
Toronto_onehot = Toronto_onehot[fixed_columns]

Toronto_onehot.head()

,Neighborhood,American Restaurant,BBQ Joint,Bagel Shop,Breakfast Spot,Brewery,Burger Joint,Bus Line,Café,Chinese Restaurant,Clothing Store,Coffee Shop,Cosmetics Shop,Dance Studio,Dessert Shop,Diner,Discount Store,Farmers Market,Flower Shop,Food & Drink Shop,Fried Chicken Joint,Garden,Gift Shop,Gourmet Shop,Greek Restaurant,Gym,Gym / Fitness Center,Health & Beauty Service,History Museum,Hotel,Ice Cream Shop,Indian Restaurant,Indoor Play Area,Italian Restaurant,Japanese Restaurant,Jewelry Store,Jewish Restaurant,Light Rail Station,Liquor Store,Mexican Restaurant,Music Venue,Park,Pet Store,Pharmacy,Pizza Place,Playground,Pub,Rental Car Location,Restaurant,Salon / Barbershop,Sandwich Place,Seafood Restaurant,Spa,Sporting Goods Shop,Sports Bar,Supermarket,Sushi Restaurant,Swim School,Tennis Court,Thai Restaurant,Toy / Game Store,Trail,Transportation Service,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Yoga Studio
0,"Lawrence Park, Lawrence Park",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,"Lawrence Park, Lawrence Park",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
2,"Lawrence Park, Lawrence Park",0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,"Davisville North, Davisville North",0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,"Davisville North, Davisville North",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [93]:
# grouping neighborhood by taking mean of the frequency of occurence of each category

toronto_grouped = Toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,American Restaurant,BBQ Joint,Bagel Shop,Breakfast Spot,Brewery,Burger Joint,Bus Line,Café,Chinese Restaurant,Clothing Store,Coffee Shop,Cosmetics Shop,Dance Studio,Dessert Shop,Diner,Discount Store,Farmers Market,Flower Shop,Food & Drink Shop,Fried Chicken Joint,Garden,Gift Shop,Gourmet Shop,Greek Restaurant,Gym,Gym / Fitness Center,Health & Beauty Service,History Museum,Hotel,Ice Cream Shop,Indian Restaurant,Indoor Play Area,Italian Restaurant,Japanese Restaurant,Jewelry Store,Jewish Restaurant,Light Rail Station,Liquor Store,Mexican Restaurant,Music Venue,Park,Pet Store,Pharmacy,Pizza Place,Playground,Pub,Rental Car Location,Restaurant,Salon / Barbershop,Sandwich Place,Seafood Restaurant,Spa,Sporting Goods Shop,Sports Bar,Supermarket,Sushi Restaurant,Swim School,Tennis Court,Thai Restaurant,Toy / Game Store,Trail,Transportation Service,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Yoga Studio
0,"Davisville North, Davisville North",0.000000,0.000000,0.000000,0.125,0.000000,0.000000,0.000000,0.000000,0.000000,0.125000,0.000000,0.000000,0.125,0.000000,0.000000,0.000000,0.000000,0.000000,0.125,0.000000,0.0,0.000000,0.000000,0.000000,0.125000,0.000000,0.000000,0.000000,0.125,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.0,0.125000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.125000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000
1,"Davisville, Davisville",0.025641,0.000000,0.000000,0.000,0.025641,0.000000,0.000000,0.051282,0.000000,0.000000,0.051282,0.000000,0.000,0.076923,0.025641,0.025641,0.025641,0.025641,0.000,0.025641,0.0,0.000000,0.025641,0.025641,0.051282,0.000000,0.000000,0.000000,0.000,0.000000,0.025641,0.025641,0.051282,0.025641,0.00,0.000000,0.000000,0.000000,0.000000,0.0,0.025641,0.000000,0.025641,0.076923,0.000000,0.000000,0.000000,0.025641,0.000000,0.076923,0.025641,0.000000,0.000000,0.000000,0.000000,0.051282,0.000000,0.000000,0.051282,0.025641,0.00,0.025641,0.000000,0.000000,0.000000
2,"Deer Park, Forest Hill SE, Rathnelly, South Hi...",0.066667,0.000000,0.066667,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.133333,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.066667,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.066667,0.066667,0.000000,0.0,0.000000,0.000000,0.000000,0.066667,0.000000,0.133333,0.000000,0.066667,0.000000,0.000000,0.000000,0.000000,0.000000,0.066667,0.066667,0.066667,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.066667,0.000000
3,"Forest Hill North, Forest Hill West, Forest Hi...",0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.25,0.000000,0.000000,0.000000,0.000000,0.0,0.250000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.250000,0.000000,0.000000,0.000000,0.000000,0.25,0.000000,0.000000,0.000000,0.000000
4,"Lawrence Park, Lawrence Park",0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.333333,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.0,0.333333,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.333333,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000
5,"Moore Park, Summerhill East, Moore Park, Summe...",0.000000,0.000000,0.000000,0.000,0.

In [94]:
toronto_grouped.shape

(9, 66)

In [95]:
# top 5 venues of each toronto neighborhood

num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Davisville North, Davisville North----
            venue  freq
0  Clothing Store  0.12
1           Hotel  0.12
2             Gym  0.12
3    Dance Studio  0.12
4  Sandwich Place  0.12


----Davisville, Davisville----
                venue  freq
0        Dessert Shop  0.08
1         Pizza Place  0.08
2      Sandwich Place  0.08
3  Italian Restaurant  0.05
4                Café  0.05


----Deer Park, Forest Hill SE, Rathnelly, South Hill, Summerhill West, Deer Park, Forest Hill SE, Rathnelly, South Hill, Summerhill West----
         venue  freq
0  Coffee Shop  0.13
1          Pub  0.13
2   Restaurant  0.07
3  Pizza Place  0.07
4   Sports Bar  0.07


----Forest Hill North, Forest Hill West, Forest Hill North, Forest Hill West----
              venue  freq
0     Jewelry Store  0.25
1             Trail  0.25
2  Sushi Restaurant  0.25
3              Park  0.25
4      Liquor Store  0.00


----Lawrence Park, Lawrence Park----
                 venue  freq
0                 Park  0.33
1      

In [98]:
#sorting the venues in order and create data frame

def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]


num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues

columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Davisville North, Davisville North",Food & Drink Shop,Park,Hotel,Breakfast Spot,Sandwich Place,Gym,Dance Studio,Clothing Store,Fried Chicken Joint,Garden
1,"Davisville, Davisville",Pizza Place,Sandwich Place,Dessert Shop,Thai Restaurant,Gym,Sushi Restaurant,Italian Restaurant,Coffee Shop,Café,Diner
2,"Deer Park, Forest Hill SE, Rathnelly, South Hi...",Pub,Coffee Shop,American Restaurant,Sushi Restaurant,Restaurant,Fried Chicken Joint,Liquor Store,Light Rail Station,Supermarket,Sports Bar
3,"Forest Hill North, Forest Hill West, Forest Hi...",Sushi Restaurant,Trail,Jewelry Store,Park,Gourmet Shop,Flower Shop,Food & Drink Shop,Fried Chicken Joint,Garden,Gift Shop
4,"Lawrence Park, Lawrence Park",Swim School,Park,Bus Line,Greek Restaurant,Food & Drink Shop,Fried Chicken Joint,Garden,Gift Shop,Gourmet Shop,Yoga Studio


In [107]:
# Clustering neighborhoods

# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([2, 2, 2, 3, 4, 0, 2, 1, 2], dtype=int32)

In [110]:
neighborhoods_venues_sorted.insert(0, 'Cluster Lable', kmeans.labels_)

Central_Toronto_merged = Central_Toronto_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
Central_Toronto_merged = Central_Toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

Central_Toronto_merged.head() # check the last columns!

,Postalcode,Borough,Neighborhood,Latitude,Longitude,Cluster Lable,Cluster Labels1,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4N,Central Toronto,"Lawrence Park, Lawrence Park",43.728020,-79.388790,4,4,4,Swim School,Park,Bus Line,Greek Restaurant,Food & Drink Shop,Fried Chicken Joint,Garden,Gift Shop,Gourmet Shop,Yoga Studio
1,M4P,Central Toronto,"Davisville North, Davisville North",43.712751,-79.390197,2,2,6,Food & Drink Shop,Park,Hotel,Breakfast Spot,Sandwich Place,Gym,Dance Studio,Clothing Store,Fried Chicken Joint,Garden
2,M4R,Central Toronto,"North Toronto West, North Toronto West",43.715383,-79.405678,2,2,5,Coffee Shop,Sporting Goods Shop,Yoga Studio,Salon / Barbershop,Ice Cream Shop,Gym / Fitness Center,Gift Shop,Mexican Restaurant,Park,Pet Store
3,M4S,Central Toronto,"Davisville, Davisville",43.704324,-79.388790,2,2,5,Pizza Place,Sandwich Place,Dessert Shop,Thai Restaurant,Gym,Sushi Restaurant,Italian Restaurant,Coffee Shop,Café,Diner
4,M4T,Central Toronto,"Moore Park, Summerhill East, Moore Park, Summe...",43.689574,-79.383160,0,0,0,Playground,Gym,Tennis Court,Yoga Studio,Gift Shop,Flower Shop,Food & Drink Shop,Fried Chicken Joint,Garden,Greek Restaurant


In [112]:
# Visualzing the cluster


map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(Central_Toronto_merged['Latitude'], Central_Toronto_merged['Longitude'], Central_Toronto_merged['Neighborhood'], Central_Toronto_merged['Cluster Lable']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [118]:
# Examining the Cluster 0

Central_Toronto_merged.loc[Central_Toronto_merged['Cluster Lable'] == 0, Central_Toronto_merged.columns[[1] + list(range(5, Central_Toronto_merged.shape[1]))]]


,Borough,Cluster Lable,Cluster Labels1,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4,Central Toronto,0,0,0,Playground,Gym,Tennis Court,Yoga Studio,Gift Shop,Flower Shop,Food & Drink Shop,Fried Chicken Joint,Garden,Greek Restaurant


In [119]:
# Examining the Cluster 1

Central_Toronto_merged.loc[Central_Toronto_merged['Cluster Lable'] == 1, Central_Toronto_merged.columns[[1] + list(range(5, Central_Toronto_merged.shape[1]))]]

,Borough,Cluster Lable,Cluster Labels1,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
6,Central Toronto,1,1,1,Music Venue,Garden,Yoga Studio,Farmers Market,Ice Cream Shop,Hotel,History Museum,Health & Beauty Service,Gym / Fitness Center,Gym


In [120]:
# Examining the Cluster 2

Central_Toronto_merged.loc[Central_Toronto_merged['Cluster Lable'] == 2, Central_Toronto_merged.columns[[1] + list(range(5, Central_Toronto_merged.shape[1]))]]

,Borough,Cluster Lable,Cluster Labels1,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Central Toronto,2,2,6,Food & Drink Shop,Park,Hotel,Breakfast Spot,Sandwich Place,Gym,Dance Studio,Clothing Store,Fried Chicken Joint,Garden
2,Central Toronto,2,2,5,Coffee Shop,Sporting Goods Shop,Yoga Studio,Salon / Barbershop,Ice Cream Shop,Gym / Fitness Center,Gift Shop,Mexican Restaurant,Park,Pet Store
3,Central Toronto,2,2,5,Pizza Place,Sandwich Place,Dessert Shop,Thai Restaurant,Gym,Sushi Restaurant,Italian Restaurant,Coffee Shop,Café,Diner
5,Central Toronto,2,2,2,Pub,Coffee Shop,American Restaurant,Sushi Restaurant,Restaurant,Fried Chicken Joint,Liquor Store,Light Rail Station,Supermarket,Sports Bar
8,Central Toronto,2,2,5,Sandwich Place,Café,Coffee Shop,Park,Pharmacy,Pizza Place,Pub,History Museum,Liquor Store,Jewish Restaurant


In [121]:
# Examining the Cluster 3

Central_Toronto_merged.loc[Central_Toronto_merged['Cluster Lable'] == 3, Central_Toronto_merged.columns[[1] + list(range(5, Central_Toronto_merged.shape[1]))]]

,Borough,Cluster Lable,Cluster Labels1,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
7,Central Toronto,3,3,3,Sushi Restaurant,Trail,Jewelry Store,Park,Gourmet Shop,Flower Shop,Food & Drink Shop,Fried Chicken Joint,Garden,Gift Shop


In [122]:
# Examining the Cluster 4

Central_Toronto_merged.loc[Central_Toronto_merged['Cluster Lable'] == 4, Central_Toronto_merged.columns[[1] + list(range(5, Central_Toronto_merged.shape[1]))]]

,Borough,Cluster Lable,Cluster Labels1,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Central Toronto,4,4,4,Swim School,Park,Bus Line,Greek Restaurant,Food & Drink Shop,Fried Chicken Joint,Garden,Gift Shop,Gourmet Shop,Yoga Studio
